# RNA-Seq Analysis Training Demo

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a prokaryotic data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression.

![RNA-Seq workflow](images/rnaseq-workflow.png)

### STEP 1: Setup Environment
We to create a set of directories first.


Set up directory structure

In [1]:
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc
!mkdir -p data/aligned
!mkdir -p data/reference

### STEP 2: Copy FASTQ Files

In [2]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349122_1.fastq --output data/raw_fastq/SRR13349122_1.fastq
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349122_2.fastq --output data/raw_fastq/SRR13349122_2.fastq
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349123_1.fastq --output data/raw_fastq/SRR13349123_1.fastq
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349123_2.fastq --output data/raw_fastq/SRR13349123_2.fastq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7175k  100 7175k    0     0  13.7M      0 --:--:-- --:--:-- --:--:-- 13.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7175k  100 7175k    0     0  12.9M      0 --:--:-- --:--:-- --:--:-- 12.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7178k  100 7178k    0     0  13.3M      0 --:--:-- --:--:-- --:--:-- 13.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7178k  100 7178k    0     0  6342k      0  0:00:01 --:--:--  0:00:01 6327k    0     0  13.4M      0 --:--:-- --:--:-- --:--:-- 

### STEP 3: Copy reference transcriptome files

In [3]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_NZ_CP007220.fasta --output data/reference/M_chelonae_NZ_CP007220.fasta
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_NZ_CP007220.fasta.amb --output data/reference/M_chelonae_NZ_CP007220.fasta.amb
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_NZ_CP007220.fasta.ann --output data/reference/M_chelonae_NZ_CP007220.fasta.ann
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_NZ_CP007220.fasta.bwt --output data/reference/M_chelonae_NZ_CP007220.fasta.bwt
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_NZ_CP007220.fasta.fai --output data/reference/M_chelonae_NZ_CP007220.fasta.fai
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_NZ_CP007220.fasta.pac --output data/reference/M_chelonae_NZ_CP007220.fasta.pac
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_NZ_CP007220.fasta.sa --output data/reference/M_chelonae_NZ_CP007220.fasta.sa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4982k  100 4982k    0     0  10.0M      0 --:--:-- --:--:-- --:--:-- 10.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    12  100    12    0     0     80      0 --:--:-- --:--:-- --:--:--     03      0 --:--:-- --:--:-- --:--:--    83
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   105  100   105    0     0    700      0 --:--:-- --:--:-- --:--:--   700
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4912k  100 4912k    0     0  10.1M      0 --:--:-- --:--:-- --:--:-- 10.0M
  % Total  

### STEP 4: Copy data file for Trimmomatic

In [4]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa --output TruSeq3-PE.fa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    95  100    95    0     0    605      0 --:--:-- --:--:-- --:--:--   608


### STEP 5: Run Trimmomatic

In [5]:
!trimmomatic PE -threads 2 data/raw_fastq/SRR13349122_1.fastq data/raw_fastq/SRR13349122_2.fastq data/trimmed/SRR13349122_1_trimmed.fastq data/trimmed/SRR13349122_2_trimmed.fastq data/trimmed/SRR13349122_1_trimmed_unpaired.fastq  data/trimmed/SRR13349122_2_trimmed_unpaired.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
!trimmomatic PE -threads 2 data/raw_fastq/SRR13349123_1.fastq data/raw_fastq/SRR13349123_2.fastq data/trimmed/SRR13349123_1_trimmed.fastq data/trimmed/SRR13349123_2_trimmed.fastq data/trimmed/SRR13349123_1_trimmed_unpaired.fastq  data/trimmed/SRR13349123_2_trimmed_unpaired.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/SRR13349122_1.fastq data/raw_fastq/SRR13349122_2.fastq data/trimmed/SRR13349122_1_trimmed.fastq data/trimmed/SRR13349122_2_trimmed.fastq data/trimmed/SRR13349122_1_trimmed_unpaired.fastq data/trimmed/SRR13349122_2_trimmed_unpaired.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 50000 Both Surviving: 49926 (99.85%) Forward Only Surviving: 74 (0.15%) Reverse Only Surviving: 0 (0.00%) Dropped: 0 (0.00%)
TrimmomaticPE: Completed successfully
TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/SRR13349123_1.fastq data/raw_fastq/SRR13349123_2.fastq data/trimmed/SRR13349123_1_trimmed.fastq data/trimmed/SRR1334912

### STEP 6: Run FastQC

In [6]:
!fastqc -o data/fastqc data/trimmed/SRR13349122_1_trimmed.fastq
!fastqc -o data/fastqc data/trimmed/SRR13349122_2_trimmed.fastq
!fastqc -o data/fastqc data/trimmed/SRR13349123_1_trimmed.fastq
!fastqc -o data/fastqc data/trimmed/SRR13349123_2_trimmed.fastq


Started analysis of SRR13349122_1_trimmed.fastq
Approx 5% complete for SRR13349122_1_trimmed.fastq
Approx 10% complete for SRR13349122_1_trimmed.fastq
Approx 15% complete for SRR13349122_1_trimmed.fastq
Approx 20% complete for SRR13349122_1_trimmed.fastq
Approx 25% complete for SRR13349122_1_trimmed.fastq
Approx 30% complete for SRR13349122_1_trimmed.fastq
Approx 35% complete for SRR13349122_1_trimmed.fastq
Approx 40% complete for SRR13349122_1_trimmed.fastq
Approx 45% complete for SRR13349122_1_trimmed.fastq
Approx 50% complete for SRR13349122_1_trimmed.fastq
Approx 55% complete for SRR13349122_1_trimmed.fastq
Approx 60% complete for SRR13349122_1_trimmed.fastq
Approx 65% complete for SRR13349122_1_trimmed.fastq
Approx 70% complete for SRR13349122_1_trimmed.fastq
Approx 75% complete for SRR13349122_1_trimmed.fastq
Approx 80% complete for SRR13349122_1_trimmed.fastq
Approx 85% complete for SRR13349122_1_trimmed.fastq
Approx 90% complete for SRR13349122_1_trimmed.fastq
Approx 95% comple

### STEP 7: Run MultiQC

In [7]:
!multiqc data/fastqc


[WARNING]         multiqc : MultiQC Version v1.11 now available!
[INFO   ]         multiqc : This is MultiQC v1.10.1
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /home/jovyan/data/fastqc
Searching   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 8/8  data/fastqc/SRR13349123_1_trimmed_fastqc.html
[INFO   ]          fastqc : Found 4 reports
[INFO   ]         multiqc : Compressing plot data
[INFO   ]         multiqc : Report      : multiqc_report.html
[INFO   ]         multiqc : Data        : multiqc_data
[INFO   ]         multiqc : MultiQC complete


### You are all done!